# Recurrent Neural Networks

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 28/02/25  | Martin | Created   | Create for chapter 9. Started on text generation section | 

# Content

* [Introduction](#introduction)
* [Text Generation - LSTM](#text-generation)

# Introduction

__Recurrent Neural Networks (RNN)__ model data that is sequential in nature. Recurrent refers to data where the output of the current step becomes the input to the next one. At each step, the model considers what it has seen about the preceding elements on top of the current input.

__Natural Language Processing (NLP)__ is where we train models to understand text information by training them on the context

Topics covered:

1. Text generation
2. Sentiment classification
3. Time series - stock information
4. Open-domain question answering

---

# Text Generation

Use a _Long Short-Term Memory (LSTM)_ architecture to build a text generation model

* Standard RNN models suffer from long dependencies i.e words that are earlier in the context window no longer contribute to the model since they're further away (vanishing gradient problem).
* LSTM maintains a cell state, and a "carry" to ensure the signal is not loss as the sequence progresses
* Each step: (1) current word (2) carry (3) cell state

Video References

* [RNN Explained](https://www.youtube.com/watch?v=AsNTP8Kwu80&ab_channel=StatQuestwithJoshStarmer)
* [LSTM Explained](https://www.youtube.com/watch?v=YCzL96nL7j0&ab_channel=StatQuestwithJoshStarmer)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

# Keras modules for LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Emdedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

import numpy as np
np.random.seed(7)
tf.random.set_seed(7)
import random
random.seed(7)
tf.random.unifrom([1], seed=1)

import pandas as pd
import string

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "2"

In [ ]:
# Define functions that simplify the workflow
def clean_text(txt):
  """
  Removes punctuations and lowercase text.
  Then convert text into utf-8 format
  """
  txt = "".join(v for v in text if v not in string.punctuation).lower()
  txt = txt.encode('utf8').decode('ascii', 'ignore')
  return txt


def get_sequence_of_tokens(corpus):
  """

  """
  # Tokeniser
  tokenizer.fit_on_text(corpus)
  total_words = len(tokenizer.word_index) + 1

  # Convert data to sequence of tokens
  input_sequence = []
  for line in corpus:
    token_list = tokenizer.text_to_sequences([line])[0]
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)
  return input_sequence, total_words


def generate_padded_sequences(input_sentences):
  """
  1. Ensure that all the sequenes are of the same length by adding padding
  All padding is added to the front
  2. Separate the predictions (text content) and labels (last word in the sequence)
  3. Convert the label into a categorical variable. Categories are all available words
  in the corpus
  """
  max_sequence_len = max([len(x) for x in input_sentences])
  input_sequences = np.array(pad_sequences(
    input_sequences,
    maxlen=max_sequence_len,
    padding='pre'
  ))

  predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
  label = ku.to_categorical(label, num_classes=total_words)
  return predictors, label, max_sequence_len


def generate_text(seed_text, next_words, model, max_sequence_len):
  """
  1. Add the same preprocessing done to the text
  2. Make predictions of next word
  3. Add the predicted word to the end of the seed text
  """
  for _ in range(next_words):
    # Apply the same preprocessing as the model
    token_list = tokenizer.text_to_sequences([seed_text])[0]
    token_list = pad_sequences(
      [token_list],
      maxlen=max_sequence_len-1, # need to remove the label
      padding='pre'
    )

    # Make a prediction on the next word
    predicted = model.predict_classes(token_list, verbose=0)

    # Convert the prediction back to actual word
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
      
    seed_text += " " + output_word
  
  return seed_text.title()

In [ ]:
def create_model(max_sequence_len, total_words):
  """
  Create model with single LSTM hidden layer
  """
  input = max_sequence_len - 1
  model = Sequential()

  # Embedding layer
  model.add(Embedding(total_words, 10, input_length=input_len))

  # LSTM layer
  model.add(LSTM(100))

  # Output layer
  model.add(Dense(total_words, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam')

  return model